## This notebook runs the S diffusion model from Wieser et al. (bulletin of Volcanology)

In [ ]:
# %% Import Python Libraries used for calculations.
import numpy as np
import sympy as sp

from matplotlib import rc
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Set plotting parameters
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'retina'")
plt.rcParams.update({
    'xtick.major.size': 4,
    'ytick.major.size': 4,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'axes.titlesize': 16,
    'axes.labelsize': 16,
    'pdf.fonttype': 42,
    'font.family': 'Avenir',
    'font.size': 13,
    'xtick.direction': 'in',  # Set x-tick direction to 'in'
    'ytick.direction': 'in',  # Set y-tick direction to 'in'
    'xtick.major.size': 5,    # Set x-tick length
    'ytick.major.size': 5,    # Set y-tick length
    'xtick.major.pad': 6.5,   # Set x-tick padding
    'ytick.major.pad': 6.5    # Set y-tick padding
})

In [ ]:
# This is function that actually runs the diffusion calculation
def run_diffusion_spherical(dr, D, radius_melt, radius_fluid_inclusion,
                            S_melt_init, S_FI_init,
                            KD, total_time=600, save_interval=1):
    """
    Simulate sulfur diffusion in spherical coordinates with a fluid inclusion and mass balance.

    Parameters:
        dr : step size (m)
        D (float): Diffusivity (m^2/s).
        radius_melt (float): Radius of the melt (m).
        radius_melt_film (float): Radius of the melt film (m).
        radius_fluid_inclusion (float): Radius of the spherical fluid inclusion (m).
        S_melt_init (float): Initial sulfur concentration in the melt / melt film (ppm).
        S_FI_init (float): Initial sulfur concentration in the fluid inclusion (ppm).
        mass_FI (float): Mass of the fluid inclusion (kg).
        mass_melt (float): Mass of the melt film (kg).
        KD (float): Partition coefficient.
        total_time (float): Total simulation time in seconds (default: 600s).
    """
    
    ### Step 1: Setup grid and initial parameters
    
    radius_melt_film = dr
    total_radius = (radius_melt) #+ radius_melt_film + radius_fluid_inclusion)
    r = np.arange(0, total_radius + dr, dr)  # Radial grid
    Nr = len(r)  # Number of radial points
    dt = (0.25 * dr**2) / D  # Time step size based on CFL condition
    time_steps = int(total_time / dt)  # Number of time steps
    save_interval = save_interval  # Frequency of saving the results

    # Define density 
    density_melt = 2700 # kg/m^3
    density_FI = 160 # kg/m^3

    # Calculate the mass of these elements in 3D
    constant = 4/3 * np.pi
    volume_FI = constant * radius_fluid_inclusion**3
    surface_FI = constant * 3 * radius_fluid_inclusion**2
    volume_melt = constant * (radius_melt_film+radius_fluid_inclusion)**3 - volume_FI

    # mass_melt_all = density_melt * volume_melt
    mass_FI = density_FI * volume_FI
    mass_SO2 = 0.0 # initial mass of S in the system

    # Initialize concentration array
    C = np.full(Nr, S_melt_init, dtype=float)  # Initial concentration in melt

    # print('Initial mass balance completed')

    #print('Initial:', round(S_mf_initial, 4), round(S_FI_initial, 4), X_FI_initial, X_mf_initial)

    ### Step 2: Define masks for geometry
    fluid_inclusion_mask = r <= radius_fluid_inclusion
    melt_film_mask = (r > radius_fluid_inclusion) & (r <= radius_fluid_inclusion + radius_melt_film)
    melt_mask = r > radius_fluid_inclusion + radius_melt_film
    interface_index = np.argmax(r > radius_fluid_inclusion)
    
    C[fluid_inclusion_mask] = 0.0 #S_FI_initial  # Initial concentration in fluid inclusion r <= radius_fluid_inclusion
    C[melt_film_mask] = S_melt_init #S_mf_initial  # Melt film is set to the mass balance value (~65.94 ppm)

    ### Step 3: Initialize storage for outputs
    C_profile = np.zeros((time_steps // save_interval + 1, Nr))
    C_profile[0, :] = C
    S_FI_over_time = np.zeros(time_steps)
    S_mf_over_time = np.zeros(time_steps)

    print(time_steps)
    ### Step 4: Diffusion and mass balance loop
    for t in range(time_steps):
        
        if t==0:
            S_FI_step_prior=S_FI_init
        else:
            S_FI_step_prior=S_FI_step_calc

        try:
            # Step 4.1: Update diffusion in spherical coordinates
            C_new = C.copy()
            for i in range(1, Nr - 1):  # Skip boundaries
                if i == interface_index:
                    # The grid cell i is the first point in the melt

                    # first calculate flux across the boundary
                    J = D*(C[i] - S_FI_step_prior/KD)/dr

                    # Calculate S loss from the melt film due to flux into the fluid
                    dS = -(J*surface_FI*dt)/(volume_melt) # change in melt conc due to loss to the bubble

                    # combine this with Fick's second law to calculate change in the melt film concentration
                    dC_dr = (C[i+1] - C[i]) / dr
                    d2C_dr2 = (C[i+1] - C[i]) / dr**2
                    C_new[i] = C[i] + D * dt * (d2C_dr2 + (2 / r[i]) * dC_dr) + dS

                    # calculate true mass transfer to the fluid (ds) accounting for differences in densities
                    dS_fluid = -dS*volume_melt*density_melt*10**(-6) # kg of S

                    # work out new masses of FI and SO2 in FI
                    mass_FI = mass_FI + 2*dS_fluid # adding the mass of SO2 to the fluid
                    mass_SO2 = mass_SO2 + 2*dS_fluid

                    # calculate S content in ppm
                    S_FI_step_calc = ((0.5*mass_SO2)/mass_FI)*10**6
                    S_mf_diffusion = C_new[i]
                    
                else:
                    # For all other points use the central difference.
                    dC_dr = (C[i+1] - C[i-1]) / (2*dr)
                    d2C_dr2 = (C[i+1] - 2 * C[i] + C[i-1]) / dr**2
                    C_new[i] = C[i] + D * dt * (d2C_dr2 + (2 / r[i]) * dC_dr)

            # Step 4.2: Apply boundary conditions
            C_new[0] = C_new[1]  # Symmetry condition at r = 0
            C_new[-1] = C_new[-2]  # No-flux condition at outer boundary
            C = C_new

            print('S in melt film=', round(S_mf_diffusion, 4), 'S in F inc=', round(S_FI_step_calc))

            # Step 4.6: Update sulfur concentrations
            S_FI_over_time[t] = S_FI_step_calc  # Track sulfur in the fluid inclusion
            S_mf_over_time[t] = S_mf_diffusion  # Track sulfur in the melt film
            C[fluid_inclusion_mask] = S_FI_step_calc  # Update fluid inclusion concentration

            # Step 4.7: Save the concentration profile at intervals
            if t % save_interval == 0:
                C_profile[t // save_interval, :] = C

        except ValueError as e:
            # Save partial results and break
            print(f"Error at time {t * dt} seconds: {e}")
            break

    ### Step 5: Return results even if terminated early
    return C_profile[:t // save_interval + 1], S_FI_over_time[:t], S_mf_over_time[:t], dr, dt, save_interval


In [ ]:
# %% Set the parameters for the diffusion model here, which are then fed into the function above

dr = 0.2*1e-6
D = 1e-12  # Diffusivity (m^2/s)
radius_melt = 200*1e-6  # Radius of the melt (m)
radius_fluid_inclusion = 8*1e-6  # Radius of the fluid inclusion (m)
S_melt_init = 1400  # Sulfur concentration in the melt (ppm)
S_FI_init = 0 # Sulfur concentration in the fluid inclusion (ppm)
total_radius = (radius_melt) # + dr + radius_fluid_inclusion)
KD=100

C_profile, S_FI_over_time, S_mf_over_time, dr, dt, save_interval = run_diffusion_spherical(dr=dr, D=D, radius_melt=radius_melt, radius_fluid_inclusion=radius_fluid_inclusion,
                                                                                           S_melt_init=S_melt_init, S_FI_init=S_FI_init,  
                                                                                           KD=KD, total_time=1200)


In [ ]:
# now lets save the results. 
import os

# Create the directory if it doesn't exist
os.makedirs('results', exist_ok=True)
np.savez(f'results/spherical_C_profile_dr{dr}.npz', C_profile=C_profile, S_FI_over_time=S_FI_over_time, S_mf_over_time=S_mf_over_time, dr=dr, dt=dt, save_interval=save_interval)
np.savez(f'results/spherical_S_results_dr{dr}.npz', S_FI_over_time=S_FI_over_time, S_mf_over_time=S_mf_over_time, dr=dr, dt=dt, save_interval=save_interval)


In [ ]:
def convert_s_ppm_to_so2_molperc(S_ppm):
    """
    Convert sulfur concentration (ppm) to SO2 mol% for scalar or array input.

    Parameters:
        S_ppm: Scalar or numpy array of sulfur concentrations in ppm.
    Returns:
        mol_frac_SO2: Scalar or numpy array of SO2 mol% values.
    """
    # Ensure input is a NumPy array for element-wise operations
    S_ppm = np.asarray(S_ppm)
    
    # Convert ppm to weight percent
    S_wt = S_ppm / 10**4
    # Calculate moles of S
    S_moles = S_wt / 32.065
    # Moles of O from SO2
    O_moles_from_SO2 = S_moles * 2
    # Oxygen weight contribution from SO2
    O_wt_from_SO2 = O_moles_from_SO2 * 16
    # Remaining mass as CO2
    mass_CO2 = 100 - S_wt - O_wt_from_SO2
    # Moles of CO2
    moles_CO2 = mass_CO2 / 44.009
    # Mole fraction of SO2
    mol_frac_SO2 = S_moles / (S_moles + moles_CO2)
    
    return 100*mol_frac_SO2

In [ ]:

# This plots the results. 

timestep = np.linspace(0, dt*len(S_FI_over_time) - dt,len(S_FI_over_time))
C_profile_molperc = np.copy(C_profile)  # Create a copy of C_profile
r = np.arange(0, radius_melt + dr, dr)  # Radial grid
fluid_inclusion_mask = r <= radius_fluid_inclusion  # Mask for the fluid inclusion region
C_profile_molperc[:, fluid_inclusion_mask] = convert_s_ppm_to_so2_molperc(C_profile[:, fluid_inclusion_mask])

plt.subplots_adjust(wspace=None, hspace=None)
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = (6,5), sharex=True, gridspec_kw={'hspace': 0.0})

# -----------------------------
# 1D Radial Transect Setup
# -----------------------------
# Create a twin y-axis so that we can plot two different datasets.
ax1_twin = ax1.twinx()
ax2_twin = ax2.twinx()
ax3_twin = ax3.twinx()

# Mask the fluid inclusion region in C_profile with NaN
fluid_inclusion_mask = r <= radius_fluid_inclusion  # Mask for the fluid inclusion region
C_profile_masked = np.copy(C_profile)
C_profile_masked[:, fluid_inclusion_mask] = np.nan 

# Mask the melt region in C_profile_molperc with NaN
C_profile_molperc_masked = np.copy(C_profile_molperc)
C_profile_molperc_masked[:, ~fluid_inclusion_mask] = np.nan  # Mask melt region
t = [10, 50, 180]

idx = np.argmin(np.abs(timestep-t[0]))
line1, = ax1.plot(r * 1e6, C_profile_masked[idx, :], label=f"S (ppm)", color='#E42211')

# Plot the fluid inclusion profile (blue dashed line) on the twin axis.
line2, = ax1_twin.plot(r * 1e6, C_profile_molperc_masked[idx, :], label='$\\mathregular{{S_{{FI}}}}$ (mol%)', 
                    color='#0A306B', linestyle='--', linewidth=3)

# ax1.set_xlabel('Radius (µm)')
ax1.set_ylabel('S$_{melt}$ (ppm)', color='#E42211')
ax1_twin.set_ylabel('$\\mathregular{{S_{{FI}}}}$ (mol%)', color='#0A306B')
ax1.set_xlim([r[-1]*1e6, 0])
ax1.set_ylim([np.nanmin(C_profile_masked)*0.975, np.nanmax(C_profile_masked)*1.125])
ax1_twin.set_ylim([0, np.nanmax(C_profile_molperc_masked)*1.125])

idx = np.argmin(np.abs(timestep-t[1]))
line1, = ax2.plot(r * 1e6, C_profile_masked[idx, :], label=f"S (ppm)", color='#E42211')

# Plot the fluid inclusion profile (blue dashed line) on the twin axis.
line2, = ax2_twin.plot(r * 1e6, C_profile_molperc_masked[idx, :], label='$\\mathregular{{S_{{FI}}}}$ (mol%)', 
                    color='#0A306B', linestyle='--', linewidth=3)

ax2.set_ylabel('S$_{melt}$ (ppm)', color='#E42211')
ax2_twin.set_ylabel('$\\mathregular{{S_{{FI}}}}$ (mol%)', color='#0A306B')
ax2.set_xlim([r[-1]*1e6, 0])
ax2.set_ylim([np.nanmin(C_profile_masked)*0.975, np.nanmax(C_profile_masked)*1.125])
ax2_twin.set_ylim([0, np.nanmax(C_profile_molperc_masked)*1.125])

idx = np.argmin(np.abs(timestep-t[2]))
line1, = ax3.plot(r * 1e6, C_profile_masked[idx, :], label=f"S (ppm)", color='#E42211')

# Plot the fluid inclusion profile (blue dashed line) on the twin axis.
line2, = ax3_twin.plot(r * 1e6, C_profile_molperc_masked[idx, :], label='$\\mathregular{{S_{{FI}}}}$ (mol%)', 
                    color='#0A306B', linestyle='--', linewidth=3)

ax3.set_xlabel('Radius (µm)')
ax3.set_ylabel('S$_{melt}$ (ppm)', color='#E42211')
ax3_twin.set_ylabel('$\\mathregular{{S_{{FI}}}}$ (mol%)', color='#0A306B')
ax3.set_title(f"time = {t[2]} seconds")
ax3.set_xlim([r[-1]*1e6, 0])
ax3.set_ylim([np.nanmin(C_profile_masked)*0.975, np.nanmax(C_profile_masked)*1.125])
ax3_twin.set_ylim([0, np.nanmax(C_profile_molperc_masked)*1.125])

In [ ]:
# # Create a time axis (in seconds) that matches S_FI_over_time.
time_array = np.arange(len(S_FI_over_time)) * dt
S_FI_over_time_molperc = convert_s_ppm_to_so2_molperc(S_FI_over_time)

fig, ax1 = plt.subplots(figsize=(8, 6))
# # Primary y-axis for S_mf (ppm)
ax1.plot(time_array, S_mf_over_time, lw=2, label='Sulfur in Melt Film (ppm)', color='#E42211')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('$\\mathregular{{S_{{melt film}}}}$ (ppm)', color='#E42211')
ax1.tick_params(axis='y', labelcolor='#E42211')

# Secondary y-axis for S_FI (mol%)
ax2 = ax1.twinx()
ax2.plot(time_array, S_FI_over_time_molperc, ls='-.', lw=3, label='Sulfur in Fluid Inclusion (mol%)', color='#0A306B', alpha=0.5)
ax2.set_ylabel('$\\mathregular{{S_{{FI}}}}$ (mol%)', color='#0A306B')
ax2.tick_params(axis='y', labelcolor='#0A306B')
fig.tight_layout()
plt.savefig(f"results/diffusion_over_time_{dr}_KD{KD}_radFI{radius_fluid_inclusion}_D{D}.pdf")
plt.show()